In [26]:
%pylab inline

import pandas as pd

from soln.dataset import get_dev_split
from soln.dataset import get_extended_X
from soln.dataset import load_raw_data
from soln.dataset import log_transform_y

raw = load_raw_data()
X_train, y_train, X_test, y_test = get_dev_split(raw)
X_train = get_extended_X(X_train, raw)
X_test = get_extended_X(X_test, raw)
y_train = log_transform_y(y_train)
y_test = log_transform_y(y_test)

Populating the interactive namespace from numpy and matplotlib


In [40]:
from soln.featurizer import CustomFeaturizer
featurizer = CustomFeaturizer()
featurizer.fit(X_train)
X_train_feats = featurizer.transform(X_train)
X_test_feats = featurizer.transform(X_test)
list(X_train_feats.columns)

['supplier other',
 'supplier S-0042',
 'supplier S-0005',
 'supplier S-0026',
 'supplier S-0027',
 'supplier S-0072',
 'supplier S-0062',
 'supplier S-0064',
 'supplier S-0043',
 'supplier S-0066',
 'supplier S-0041',
 'supplier S-0105',
 'supplier S-0080',
 'supplier S-0081',
 'supplier S-0104',
 'supplier S-0013',
 'supplier S-0014',
 'supplier S-0070',
 'supplier S-0018',
 'supplier S-0031',
 'supplier S-0030',
 'supplier S-0058',
 'supplier S-0054',
 'supplier S-0092',
 'material_id other',
 'material_id nan',
 'material_id SP-0046',
 'material_id SP-0041',
 'material_id SP-0033',
 'material_id SP-0048',
 'material_id SP-0034',
 'material_id SP-0035',
 'material_id SP-0036',
 'material_id SP-0037',
 'material_id SP-0030',
 'material_id SP-0019',
 'material_id SP-0008',
 'material_id SP-0038',
 'material_id SP-0039',
 'material_id SP-0029',
 'material_id SP-0028',
 'annual_usage',
 'min_order_quantity',
 'quantity',
 'diameter',
 'length',
 'num_bends',
 'bend_radius',
 'num_boss',

In [43]:
X_train_np = X_train_feats.astype(np.float).values
X_test_np = X_test_feats.astype(np.float).values
y_train_np = y_train.values
y_test_np = y_test.values

In [48]:
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

regressors = [
    DummyRegressor(strategy='constant', constant=0.0),
    DummyRegressor(strategy='mean'),
    RandomForestRegressor(n_estimators=20),
    # RandomForestRegressor(n_estimators=100),
]

for reg in regressors:
    reg.fit(X_train_np, y_train_np)
    y_train_pred = reg.predict(X_train_np)
    train_rmsle = np.sqrt(mean_squared_error(y_train_np, y_train_pred))
    y_test_pred = reg.predict(X_test_np)
    test_rmsle = np.sqrt(mean_squared_error(y_test_np, y_test_pred))
    rmsle = np.sqrt(msle)
    print "{}:".format(reg)
    print "    train RMSLE {}".format(train_rmsle)
    print "    test RMSLE {}".format(test_rmsle)
    print

DummyRegressor(constant=array(0.0), quantile=None, strategy='constant'):
    train RMSLE 2.36481494682
    test RMSLE 2.20644848894

DummyRegressor(constant=None, quantile=None, strategy='mean'):
    train RMSLE 0.823952519474
    test RMSLE 0.81837011564

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False):
    train RMSLE 0.112042432844
    test RMSLE 0.359341971297

